In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def gnCr(max_n, p=10**9+7):
  max_n = min(max_n, p - 1)

  F, iF = [0] * (max_n + 1), [0] * (max_n + 1)
  F[0] = 1
  for i in range(max_n):
    F[i+1] = F[i] * (i+1) % p
  
  iF[-1] = pow(F[-1], -1, p)
  for i in reversed(range(max_n)):
    iF[i] = iF[i+1] * (i+1) % p

  def nCr(n, r):
    if n < r: return 0
    res = 1
    while n or r:
      a, b = n % p, r % p
      if a < b: return 0
      res = res * F[a] % p
      res = res * iF[b] % p
      res = res * iF[a-b] % p
      n //= p
      r //= p
    return res
  return nCr

nCr = gnCr(4000000)
sys.stdout.write('\n'.join(map(str, [nCr(*map(int, input().split())) for _ in range(int(input()))])))

### 관찰
- 파스칼의 삼각형의 일부를 그려봤다. $a$ 번째 행과 $a+1$ 번째 행을 그려보자.
  - $a \qquad 1 \enspace x_1 \enspace x_2 \enspace x_3 \enspace x_4 \cdots$
  - $a' \quad \enspace 1 \enspace x'_1 \enspace x'_2 \enspace x'_3 \enspace x'_4 \enspace x'_5 \cdots$
  - 이때 $x'_1 = x_1 + 1$ 이고, $x'_2 = x'_1 + x_2$ 등의 패턴으로 첫째 항에서 뭔가 할 수 있을수도 있다고 생각했다.
    - 그런데, 이 방법은 이전행의 $x_2$ 부터는 값을 알 수 없다. 전부 이전 값으로부터 계산해서 나오는 답이기 때문.

### 자력솔(그리고 날먹으로 인한 풀이)
- 풀이를 적어야 하는 자력솔이다. 블랙박스인채로 둘 순 없지.
- 이항 계수 수식을 그려보자.
  - $\binom{n}{r} = \frac {n!}{r!(n-r)!}$
- 문제의 $N$ 과 $R$ 의 범위는 400만으로 제한돼있다.
  - 그렇다면 수식의 각 항 $r, n, n-r$ 을 살펴보면 모두 $n$ 보다 작다. \
  즉, 구해야 하는 팩토리얼의 개수가 많아봐야 400만이란 뜻이다.
  - 그리고 모듈러 계에서 팩토리얼을 구하는 것에는 무리가 없다 - 모듈러보다 작은 수로 유지될테니.
  - 따라서 400만 까지의 팩토리얼을 pre-caclulate 해둔다.
- 이제 모듈러계에서 $\frac {n!}{r!(n-r)!}$ 을 수행하면 된다.
  - 모듈러계에서 나눗셈은 역원을 곱하는 것으로 구현할 수 있다.
- 풀이보다 시간복잡도의 유도 과정이 더욱 흥미롭다.
  - $N$ 의 최대 범위인 `max_n` 을 $X$, 모듈러를 $p$ 라고 하자.
  - 전처리 : $O(X + \log p)$
    - 그 역원의 마지막 항의 계산은 $O(\log p)$ 으로 한 번만 구하고, 나머지는 거꾸로 구하면 선형시간 내에 구간 역원도 모두 구할 수 있다.
      - https://koosaga.com/63
      - $(n-1)!^{-1} = n \cdot n!^{-1}$
    - 팩토리얼과 그 역원의 계산은 $O(1)$, 그걸 $X$ 번 구할 예정이니 $O(X)$
  - 쿼리: $O(\log_p X)$
    - $n$ 과 $r$ 은 매번 모듈러인 $p$ 만큼 나뉘어질테니, 쿼리의 while문을 $log_p$ 라고 표현할 수 있다.
    - 두 개 모두가 0이 되면 내부 반복문이 종료되므로 $O(\log_p \max(n, r))$
    - 그런데, $n < r$ 이면 계산할 필요없이 0이다. 따라서 $O(\log_p n)$
    - 게다가, 전처리 범위 $X$ 는 사실 $n$ 의 upper bound이기도 하다. 따라서 $O(\log_p X)$
  - 최종 시간복잡도 : $O(X + \log p + M \log_p X)$